<a href="https://colab.research.google.com/github/jleandroforte/Desafio_AlgoritmoML_MVP_Forte/blob/main/Desafio_AlgoritmoML_MVP_Forte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduccion**

El dataset con el que vamos a trabajar recopila datos de ventas y actividad de los clientes en el sitio web de una farmacia online a lo largo de 90 días.

Tenemos información sobre varias características de los productos que serán detalladas en secciones siguientes, sus precios, los precios de la competencia y comportamiento de los clientes, si hacen click en un producto, si los colocan en una canasta de productos y finalmente si compran un producto. Nótese que no todas las líneas representan ventas.

La clave del dataset es que la farmacia sigue una política de 'pricing dinámico' donde los precios de cada producto son ajustados diariamente, dentro de ciertas bandas.

#**Objetivos y estrategia de estimación**

En este trabajo vamos a enfocarnos en predecir si, dadas las características cada observación o fila del dataset, se va a producir una venta o no.

Recordemos que tenemos una variable llamada Order que toma valores 0 y 1, cuando order==1, la observación representa una venta, y cuando toma el valor 0 no se trata de una venta.

Siendo que nuestra variable dependiente es categórica y toma valores 0 (no-venta) y 1 (venta) vamos a proponer un modelo de regresión logística.

En otros trabajos el foco ha estado en predecir el revenue, y se ha demostrado en la segunda pre-entrega que el modelo de regresión lineal simple no es apropiado para nuestra estructura de datos, sólo como muestra, el r2 era de
~0.5.

Estimar la probabilidad de ventas, aparte de ser una pregunta de investigación importantísima en sí misma, puede funcionar como paso intermedio para estimar el revenue. Para ver por qué, recordemos que el revenue surge de las cantidades vendidas multiplicadas por el precio. El dataset contiene los precios, y la cantidad vendida en el set de testing se puede obtener construyendo una variable dividiendo revenue por precio, de modo que, si logramos predecir correctamente las ventas, podremos calcular el revenue.


In [ ]:
# Importamos las liberías necesarias.
from google.colab import drive
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score



drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Entregas/DB'

In [ ]:
# importamos los archivos y los combinamos en un solo dataset llamado 'farmacia' a través de la variable product id.

pricing_class=pd.read_csv("class.csv", sep='|')
pricing_items=pd.read_csv('items.csv', sep='|')
pricing_train=pd.read_csv('train.csv', sep='|')

dataset = pd.merge(pricing_train, pricing_items, on='pid');
farmacia=pd.DataFrame(dataset)

In [ ]:
farmacia

In [ ]:
farmacia.shape

In [ ]:
farmacia.info

In [ ]:
farmacia.describe(include='all')

In [ ]:
farmacia.dtypes

#**Descripción de las variables:**

**Vamos a proceder a describir verbalmente nuestras variables, cuando se trate de variables que no varían en el tiempo, se explicita en la descripción**

**day**: el día que se registra, recordemos que tenemos datos 91 días de ventas y otras acciones de usuarios en el sitio web. El datatype es int.

**pid**: El id del producto, como vimos, tenemos más de 22 mil productos. El datatype es int, los productos se identifican por un numero, no por su nombre. Es una variable invariante.

**adFlag**: Nos indica si el producto en cuestión es objeto de una campaña publicitaria. El valor 1 indica que hubo capaña, y 0 indica que no la hubo.

**availability**: Status de disponibilidad de los productos, toma los valores {1,2,3,4}

**competitorPrice** : El precio de la competencia para un producto. Es un floating point.

**click, basket, order**: Denotan acciones de los usuarios, si hicieron click en un producto, si lo colocaron en un carrito de compra (pero no necesariamente lo compraron), mientras que order indica si efectivamente el registro denota una compra. Las 4 variables toman los valores {0,1}, donde 1 denota acción (compra, carrito de compra, click) y el 0 la ausencia de acción.

**price**: El precio efectivamente asociado a la observacion. Recordemos que la farmacia sigue una estrategia de 'pricing dinámico', los precios de cada producto se ajustan día a día. También es importante tener en cuenta que no se trata de precios customizados para clientes, los ajustes de precios son diarios y automáticos. Esto introduce una gran variabilidad en las observaciones que nos permitirá evaluar la influencia de los precios en las ventas, y es el insumo fundamental para predecir la demanda futura en función de como ajustemos nuestros precios. El precio es un floating point.

**revenue**: Los ingresos por ventas. Es decir, el precio multiplicado por las cantidades compradas (que no observamos, solo observamos precios, compras (la variable 'order') y revenue). El revenue es un floating point.

**manufacturer**: El fabricante de cada producto. Se identifica mediante un numero entero. Es invariante, al igual que el product id.

**group**: El grupo de productos, combina letras y numeros. Es invariante.

**content**: El contenido de un producto, se identifica bajo la nomenclatura numeroXnumero, por caso: 5X10. Es invariante.

**unit**: La unidad del producto, es un string de mayúsculas.Es invariante.

**pharmForm**: La dosis: son 3 letras mayúsculas. Es invariante.

**genericProduct**: Si se trata de un medicamento generico, toma los valores {0,1}, el 1 indica que se trata de un medicamento genérico. Es invariante.

**salesIndex**: un código de dispensión de medicamentos de Estados unidos. Es un entero. Es invariante.

**category**: categoría de negocio: es un numero de negocio. Es invariante.

**campaignIndex**: Tipo de campaña publicitaria de que fue objeto el producto, toma los valores {A,B,C}

**rrp**: El precio de referencia, recordemos que la farmacia ajusta los precios de cada producto diariamente, pero cada item tiene un precio de referencia, más adelante vamos a graficar algunos ejemplos. Es invariante, la variabilidad está en los precios efectivos.

#**Módulo de Limpieza de datos**

##**Tratamiento de Missing Values**

En primer lugar, vemos cuantos datos nos faltan por columna, y proponemos las siguientes soluciones para aquellas variables donde identificamos missing values:

Para competitor price vamos a reemplazar los missing values por la mediana del precio por cada producto, no en general, en caso de no disponer de un precio para algún producto, se reemplaza por la mediana de la variable en el dataset completo.

para pharmForm, al menos por ahora, vamos a reemplazar por el valor más frecuente, a nivel de producto y no en el agregado. No se pueden usar medidas como media o mediana porque no es una variable numérica.

Para category usamos el mismo procedimiento que para pharmForm, con las mismas consideraciones.

Para campaing index, que en el dataset toma los valores {A,B,C}, e indica el tipo de campaña publicitaria que se llevó a cabo, tenemos 2 tipos de tratamiento, dado que es dependiente de adFlag, solo cuando adFlag es igual a 1 hay campaña publicitaria, de modo que si adFlag==0, reemplazamos por "D", que es una manera de indicar que no hay campaña publicitaria, mientras que para los casos en que adFlag==1 reemplazamos por la moda o valor más repetido a nivel de producto.

In [ ]:
datos_faltantes = farmacia.isnull().sum()
print("Datos faltantes por columna: " , datos_faltantes)

Reemplazo de missing values para la variable competitorPrice, si a nivel de producto no hay una mediana, se reemplaza por la mediana de la columna en general.

In [ ]:
medianas_faltantes = farmacia.groupby('pid')['competitorPrice'].transform('median') # recordemos que pid es 'Product Id', por eso hacemos el reemplazo a ese nivel.

farmacia['competitorPrice'] = farmacia['competitorPrice'].fillna(medianas_faltantes)

# En los casos no capturados por las lineas anteriores reemplazamos por la mediana general de la variable:
mediana_competitorPrice = farmacia['competitorPrice'].median()

farmacia['competitorPrice'] = farmacia['competitorPrice'].fillna(mediana_competitorPrice)

Reemplazo de missing values para las variables pharmForm y category, estamos usando el valor más frecuente, por eso la función hace referencia a la moda ("mode")

In [ ]:
def completar_pharmform(series):
    if series.mode().empty:
        return series
    else:
        moda_pharmForm = series.mode().iloc[0]
        return series.fillna(moda_pharmForm)

farmacia['pharmForm'] = farmacia.groupby('pid')['pharmForm'].transform(completar_pharmform)

farmacia['pharmForm'].fillna('default_value', inplace=True) # esta ultima linea de codigo es para reemplazar missing values en casos no capturados por la funcion anterior.

In [ ]:
def completar_category(series):
    if series.mode().empty:
        return series
    else:
        moda_category = series.mode().iloc[0]
        return series.fillna(moda_category)

farmacia['category'] = farmacia.groupby('pid')['category'].transform(completar_category)

farmacia['category'].fillna('default_value', inplace=True) # esta ultima linea de codigo es para reemplazar missing values en casos no capturados por la funcion anterior.



Reemplazo de missing values para campaignIndex, recordemos el metodo:
tenemos 2 tipos de tratamiento, dado que es dependiente de adFlag, solo cuando adFlag es igual a 1 hay campaña publicitaria, de modo que si adFlag==0, reemplazamos por "D" (ya que las campañas son {A,B,C}, y de esta forma con la "D" podemos identificar rápidamente que no hay campaña) para los casos en que adFlag==1 reemplazamos por la moda o valor más repetido a nivel de producto.  

In [ ]:
farmacia['campaignIndex'].fillna('D', inplace=True)

mask = (farmacia['adFlag'] == 1) & (farmacia['campaignIndex'].isna())
farmacia.loc[mask, 'campaignIndex'] = farmacia[mask].groupby('pid')['campaignIndex'].transform(lambda x: x.mode().iloc[0])

farmacia['campaignIndex'].fillna('default_value', inplace=True) # esta ultima linea de codigo es para reemplazar missing values en casos no capturados por la funcion anterior.


Ahora constatamos que ya no tenemos más missing values:



In [ ]:
datos_faltantes = farmacia.isnull().sum()
print("Datos faltantes por columna: " , datos_faltantes)


##**Encoding de Variables Categoricas**

Repasemos nuestras variable categóricas y sus valores unicos para saber si son factibles de aplicar encoding.

In [ ]:
farmacia.describe(include='object')

Como vemos, tenemos 6 variables que quedemos congertir a dummies para poder usar como variables explicativas de un modelo. Como se puede observar, algunas de las variables tienen varios cientos de valores únicos, lo cual consume la totalidad de la memoria de esta versión de Colab. Vamos a limitar el encoding a las variables unit y campaignIndex.

En el caso de la variable group, recordemos que se trata de agrupamientos de productos, para los cuales tenemos la variable pid (product ID), de modo que no estariamos perdiendo informacion al excluirla de un modelo, si incluimos el product id.


Para ello usamos el metodo get_dummies

In [ ]:
dummies = pd.DataFrame(pd.get_dummies(farmacia[['unit', 'campaignIndex']]))

In [ ]:
dummies.dtypes

In [ ]:
dummies

In [ ]:
farmacia_final = pd.concat([farmacia, dummies], axis=1) #creamos un dataset que sea nuestras variables originales y las dummies.

In [ ]:
farmacia_final

In [ ]:
farmacia_final.dtypes

In [ ]:
farmacia_final.shape # Nos queda un dataset donde no tenemos pérdida de filas, e incrementamos nuestros features a 33.

#**Cambio de escala de variables numéricas continuas**

En esta sección vamos a usar el módulo scaler de scikit-learn para modificar la escala de un conjunto de variables numéricas continuas, no categóricas.

En concreto, vamos a re-escalar los precios (price), precios de referencia (rrp), precios de la competencia y revenue.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
variables_re_escaladas = ["revenue", "price", "rrp" , "competitorPrice"] # las 4 variables que vamos a re-escalar.

scaler = StandardScaler() #creamos un objeto que sea el Scaler de sklearn.

columnas_re_escaladas = scaler.fit_transform(farmacia_final[variables_re_escaladas]) #Aplicamos ("fiteamos") el scaler a nuestro set de variables de interés

dataset_re_escalado = pd.DataFrame(columnas_re_escaladas, columns=variables_re_escaladas) # Se genera un dataset con nuevas escalas para nuestras variables de interés

variables_no_re_escaladas = farmacia_final.drop(columns=variables_re_escaladas) # las restantes columnas del dataset

farmacia_re_escalada = pd.concat([dataset_re_escalado, variables_no_re_escaladas], axis=1) # Concatenamos las variables con cambio de escala con el resto.


In [ ]:
farmacia_re_escalada

#**Regresión Logistica**

##**Preparación del modelo**

Seleccionamos nuestra variable dependiente **order** y excluimos las variables categoricas que encodeamos para no generar duplicaciones.

Tambien voy a excluir el revenue como regresor, recordemos, estoy tratando de predecir si una línea va a derivar en una venta en funcion de sus caracteristicas, y el revenue por definición es precio por cantidad vendida cuando hay ventas, entonces, el revenue tiene lugar después que un cliente decide una compra, no puede ser nunca una variable explicativa de las ventas.

Tambien eliminamos lineID como variable independiente, no tiene valor explicativo

In [ ]:
X = farmacia_re_escalada.drop([ 'lineID', 'order' , 'group', 'content', 'unit', 'category','campaignIndex', 'pharmForm' , 'revenue' ], axis=1)

y = farmacia_re_escalada['order']

In [ ]:
print(X.dtypes) #volvemos a chequear que nuestras variables explicativas no son strings

In [ ]:
#Dividimos el dataset en train y test subsets, dejamos el 20% para testing.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

##**Implementamos el modelo**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg_logistica = LogisticRegression()
reg_logistica.fit(X_train, y_train)

##**Obtenemos y graficamos las predicciones del modelo y distintas medidas de accuracy**

In [ ]:
y_pred = reg_logistica.predict(X_test) # dejamos que el modelo predica los valores en el testing set.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del modelo:", f'{accuracy:.3f}')

Un accuracy del 90% es alentador, si bien no es una métrica directamente comparable comparable con el r2, recordemos de entregas anteriores que un modelo de regresión lineal redundaba en un r2 de apenas 0.5.

También debo aclarar que el re-escalamiento de variables ha redundado en una mejora del accuracy de 20 puntos, no se muestran los resultados de una regresión sin scaling para no saturar el notebook.

In [ ]:
matriz_confusion = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:\n", matriz_confusion)

La matriz de confusión nos muestra que:

> Predecimos correctamente 103.115 ventas e incorrectamente 16.384, es decir, de un total de 119.499 ventas (order==1) en el testing set, el modelo predice correctamente el 86.2%

> El modelo es aún más exitoso para predecir no-ventas, la primera columna de la matrix indica que podemos predecir el 91% de los "True Negatives", 393.925 observaciones sobre un total de 431.702 no-ventas.

In [ ]:
#Buscamos los coeficientes de la regresión y sus p-values y -scores
coeficientes = reg_logistica.coef_[0]
z_scores = reg_logistica.coef_[0] / np.std(X_train, axis=0)
p_values = np.abs(reg_logistica.coef_[0]) / np.std(X_train, axis=0)

In [ ]:
#Armamos una tabla para mostrar los coeficientes y su significatividad

regresores = X.columns
resultados = pd.DataFrame({'Coeficientes': coeficientes, 'Z-Scores': z_scores, 'P-Value': p_values})

print("Coeficientes, z_scores, y p_values del modelo :\n\n", resultados)


Del análisis de los coeficientes de la regresión quiero destacar lo siguiente:

> La relación negativa de precios, precios de la competencia y precios de referencia. Aunque debemos notar que el precio de referencia no es muy significativo con un z-score de 1.19.

> El valor positivo de adFlag, recordemos que adFlag==1 indica que un producto fue objeto de una campañana publicitaria, y esto parece influir las probabilidades de venta.

> Sigueindo con las campañas publicitarias, las campañas A y D parecen tener una relación positiva con la probabilidad de ventas, mientras que la B y la C son negativas en este modelo.

In [ ]:
#Graficamos los valores predichos por el modelo y los valores efectivos de la variable dependiente en el testing set.

plt.figure(figsize=(7, 4))
plt.hist(y_test, alpha=0.5, color='blue', label='Valor efectivo de la variable dependiente')
plt.hist(y_pred,  alpha=0.5, color='red', label='Valores predichos por el modelo')
plt.xlabel('Class')
plt.ylabel('Frecuencias')
plt.title('Distribución de valores estimados por el modelo vs los efectivos en el testing set')
plt.legend()
plt.show()

El gráfico confirma lo que se observaba en la matrix de confusión donde la mayoría, pero no la totalidad, de las observaciones del testing set son correctametne predichos por el modelo.